# Задание
## 1. Посмотреть документацию к API GitHub, разобраться как вывести список репозиториев для конкретного пользователя, сохранить JSON-вывод в файле *.json.

#### Импорт необходимых модулей

In [0]:
from pprint import pprint
import pandas as pd
import requests
import json

#### Запрос имени пользователя

In [2]:
url = 'https://api.github.com/users/'
user  = input('Введите имя пользователя\nдля получения списка его репозиториев: ')

Введите имя пользователя
для получения списка его репозиториев: docinit


#### Формирование и сохранение списка репозиториев

In [3]:
response = requests.get(f'{url}{user}/repos')
user_repos = {}
# формирование списков для создания словаря
names = []
created = []
updated = []
urls = []
# сохранение результатов успешного запроса в словарь
if response.ok:
    data = json.loads(response.text)
    for i in data:
      names.append(i['name'])
      created.append(i['created_at'])
      updated.append(i['updated_at'])
      urls.append(i['html_url'])
user_repos = {'Repository':names,'Created_at':created,'Updated_at':updated,'URL':urls}
# сохранение результатов в файл
with open('user_repos.json','w') as ur:
  json.dump(user_repos,ur)

# вывод результатов в виде
pprint(user_repos)

{'Created_at': ['2019-11-30T15:21:05Z',
                '2019-11-02T19:35:44Z',
                '2020-01-20T11:03:59Z',
                '2019-06-24T08:36:37Z'],
 'Repository': ['algorithms', 'geekbrains_db', 'geekbrains_python', 'myjobs'],
 'URL': ['https://github.com/docinit/algorithms',
         'https://github.com/docinit/geekbrains_db',
         'https://github.com/docinit/geekbrains_python',
         'https://github.com/docinit/myjobs'],
 'Updated_at': ['2019-11-30T15:42:41Z',
                '2019-11-06T07:36:56Z',
                '2020-02-19T18:11:16Z',
                '2020-02-25T15:25:08Z']}


In [4]:
# Чтение результатов из файла
pd.read_json('user_repos.json')

,Repository,Created_at,Updated_at,URL
0,algorithms,2019-11-30 15:21:05+00:00,2019-11-30 15:42:41+00:00,https://github.com/docinit/algorithms
1,geekbrains_db,2019-11-02 19:35:44+00:00,2019-11-06 07:36:56+00:00,https://github.com/docinit/geekbrains_db
2,geekbrains_python,2020-01-20 11:03:59+00:00,2020-02-19 18:11:16+00:00,https://github.com/docinit/geekbrains_python
3,myjobs,2019-06-24 08:36:37+00:00,2020-02-25 15:25:08+00:00,https://github.com/docinit/myjobs


## 2. Изучить список открытых API. Найти среди них любое, требующее авторизацию (любого типа). Выполнить запросы к нему, пройдя авторизацию. Ответ сервера записать в файл.

In [9]:
### Выбран API сайта https://betterdoctor.com/
### Запрос соберет данные по ключевому слову dental.
### Сохранятся данные о центрах / докторах, которые имеют заполненные поля адреса, телефона и информацию о центре / докторе
### Также будут сохранена частично неполная информация
realm = str(input('Введите ваш запрос: '))

Введите ваш запрос: dental


In [0]:
#### API-key for https://betterdoctor.com/ = YOUR API-key
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
url = 'https://api.betterdoctor.com/2016-03-01/doctors'
params = {
    'user_key': 'YOUR API-key',
    'name': realm,
    'limit':100
}
result = requests.get(url,params = params)
name,phone,state,city,street,street2,profile = [],[],[],[],[],[],[]
for i in result.json()['data']:
  if 'practices' in i:
    if 'name' in i['practices'][0]:
      name.append(i['practices'][0]['name'])
    else:
      name.append('Not available')
    if 'phones' in i['practices'][0]:
      phone.append(i['practices'][0]['phones'][0]['number'])
    else:
      phone.append('Not available')
    if 'state' in i['practices'][0]['visit_address']:
      state.append(i['practices'][0]['visit_address']['state'])
    else:
      state.append('')
    if 'city' in i['practices'][0]['visit_address']:
      city.append(i['practices'][0]['visit_address']['city'])
    else:
      city.append('')
    if 'street' in i['practices'][0]['visit_address']:
      street.append(i['practices'][0]['visit_address']['street'])
    else:
      street.append('')
    if 'street2' in i['practices'][0]['visit_address']:
      street2.append(i['practices'][0]['visit_address']['street2'])
    else:
      street2.append('')
    if 'bio' in i['profile']:
      profile.append(i['profile']['bio'])
    else:
      profile.append('')

In [112]:
med_centers = {'Name':name,'Phone':phone,'State': state, 'City':city, 'Street':street, 'Street2':street2, 'About': profile}
with open('med_centers.json','w') as mc:
  json.dump(med_centers,mc)
pd.DataFrame(med_centers)

,Name,Phone,State,City,Street,Street2,About
0,"Arfan Taba, DDS",9492340500,CA,San Clemente,638 Camino De Los Mares,Ste C140,"Dr. Arfan Taba, DDS, specialist in dentistry a..."
1,"Loris Abedi, DDS",8189566611,CA,Glendale,1138 N Brand Blvd,Ste A,"Dr. Loris Abedi, DDS practices dentistry at Gl..."
2,"South Texas Dental Associates, L.p.",2818723777,TX,Houston,383 Greens Rd,Ste A,"Dr. Suzanne Baldwin, DMD--specialist in dentis..."
3,"Mohamed Gouda, DDS",6618316900,CA,Bakersfield,1621 S H St,,"Dr. Yu Kim, DDS sees patients in Bakersfield, ..."
4,Premiere Dental Group,3059453354,FL,North Miami Beach,16209 NE 13th Ave,,"Dr. Axel Negron, DMD--specialist in dentistry ..."
5,"Three Lower Counties Community Services, Inc",4106515151,MD,Princess Anne,12165 Elm St,,"Dr. Sarah Dental, MD DENTALHYGIENIST treats pa..."
6,"Racha Fouad, DDS",7175341135,PA,Hershey,1135 E Chocolate Ave,Ste 101,"Dr. Racha Fouad, DDS DMD practices dentistry a..."
7,"Hwai Chang, DDS",6308109333,IL,Westmont,411 N Cass Ave,,"Dr. Hwai Chang, DDS MD, specialist in dentistr..."
8,Y.m.wu DDS & C.h.lan DDS Prof. Corp.,5107925551,CA,Fremont,4988 Paseo Padre Pkwy,# 204,"Dr. Yu-Ching Lai, DDS, specialist in orthodont..."
9,"Tarek Assi, DDS",9544376400,FL,Pembroke Pines,9001 Pembroke Rd,Ste A,"Dr. Tarek Assi, DDS DMD practices dentistry at..."
